In [1]:
import pandas as pd
import numpy as np

np.random.seed(1349)

In [8]:
df = pd.read_csv('students.csv')

In [9]:
df

,name,math,english,reading,classroom,passing_math
0,Sally,78,84,73,B,passing
1,Jane,77,79,98,B,passing
2,Suzie,96,91,91,A,passing
3,Billy,62,88,72,B,failing
4,Ada,98,88,92,B,passing
5,John,95,73,64,A,passing
6,Thomas,87,80,82,A,passing
7,Marie,99,85,94,A,passing
8,Albert,91,77,87,B,passing
9,Richard,84,61,94,A,passing


## Reshaping

We will talk about reshaping operations in more detail when we discuss tidy data, but for now we will focus on a couple of common operations that can be used to summarize our data by different subgroups.

### `pd.crosstab`

For an example of `.crosstab`, we will count the number of students passing math in each classroom.

In [12]:
#  cant do this : df.crosstab(df.passing_math, df.classroom)

In [14]:
# We will use our student grades DataFrame, df.

pd.crosstab(df.passing_math, df.classroom)

classroom,A,B
passing_math,,
failing,0,1
passing,6,5


We'll use the `pd.crosstab` function to count the number of occurances of each subgroup (i.e. each unique combination of classroom and whether or not the student is passing math):

We can also view subtotals with the `margins` set to `True`.

In [15]:
pd.crosstab(df.passing_math, df.classroom, margins=True)

classroom,A,B,All
passing_math,,,
failing,0,1,1
passing,6,5,11
All,6,6,12


The `.crosstab` function will let us view the numbers as percentages of the total as well by setting `normalize` to `True`.

In [17]:
pd.crosstab(df.passing_math, df.classroom, margins=True, normalize=True).round(3)

classroom,A,B,All
passing_math,,,
failing,0.0,0.083,0.083
passing,0.5,0.417,0.917
All,0.5,0.500,1.000


### `.pivot_table`

Here we use the `.pivot_table` method to create our summary. This method produces output similar to an excel pivot table. We must supply 3 things here:

- which values will make up the rows (the `index`)
- which values will make up the columns
- the values we are aggregating
- an aggregation method (`aggfunc`); if we can omit this, and `mean` will be used by default

For an example using the `pivot_table` method, we'll calculate the average math grade for the combination of `classroom` and `passing_math` status.

In [21]:
df.pivot_table(index='classroom', columns='passing_math', values='math', aggfunc='max')

passing_math,failing,passing
classroom,,
A,NaN,99.0
B,62.0,98.0


Here we'll create a dataframe that represents various orders at a restaurant.

In [22]:
n = 40

orders = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})

orders.sample(10)

,drink,meal
26,Water,Yakisoba Noodle
19,Water,Curry
17,Water,Curry
27,Water,Curry
7,Water,Pad Thai
2,Water,Pad Thai
29,Water,Pad Thai
33,Water,Yakisoba Noodle
37,Tea,Yakisoba Noodle
14,Tea,Yakisoba Noodle


#### `.map`

The `.map` method lets us use a dictionary to calculate the total price for an order; then I can save my calculations to a new column named `bill`. Let's do this step-by-step.

In [23]:
# Create a dictionary of prices for drinks and meals.

prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}

In [24]:
"""
Match the values in the 'drink' and 'meal' columns with the values in the 'prices' dictionary 
and perform the specified calculation. Save this calculation to a new column named 'bill'.
"""

orders['bill'] = orders.drink.map(prices) + orders.meal.map(prices)

orders.sample(10)

,drink,meal,bill
32,Tea,Pad Thai,12
27,Water,Curry,11
21,Water,Curry,11
10,Tea,Yakisoba Noodle,11
39,Water,Yakisoba Noodle,9
1,Water,Curry,11
24,Water,Curry,11
16,Water,Curry,11
34,Water,Curry,11
13,Tea,Yakisoba Noodle,11


Let's take a look at how many orders have each combination of meal and drink:

In [25]:
pd.crosstab(orders.drink, orders.meal)

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,2,5,4
Water,9,11,9


In [26]:
pd.crosstab(orders.drink, orders.meal, normalize=True, margins=True)

meal,Curry,Pad Thai,Yakisoba Noodle,All
drink,,,,
Tea,0.050,0.125,0.100,0.275
Water,0.225,0.275,0.225,0.725
All,0.275,0.400,0.325,1.000


And let's find out the average bill amount for each combination: 

In [34]:
orders.pivot_table(index='drink', columns='meal', values='bill', aggfunc = 'mean')

meal,Curry,Pad Thai,Yakisoba Noodle
drink,,,
Tea,13,12,11
Water,11,10,9


It's interesting to note that we could find the same information with a multi-level group by:

In [30]:
orders.groupby(['drink', 'meal']).bill.mean()

drink  meal           
Tea    Curry              13
       Pad Thai           12
       Yakisoba Noodle    11
Water  Curry              11
       Pad Thai           10
       Yakisoba Noodle     9
Name: bill, dtype: int64

The choice between group by and a pivot table here is mostly asthetic, and you should use whichever makes more sense to you with the problem at hand. 

### Transposing

In [31]:
df.T

,0,1,2,3,4,5,6,7,8,9,10,11
name,Sally,Jane,Suzie,Billy,Ada,John,Thomas,Marie,Albert,Richard,Isaac,Alan
math,78,77,96,62,98,95,87,99,91,84,77,83
english,84,79,91,88,88,73,80,85,77,61,91,87
reading,73,98,91,72,92,64,82,94,87,94,93,60
classroom,B,B,A,B,B,A,A,A,B,A,A,B
passing_math,passing,passing,passing,failing,passing,passing,passing,passing,passing,passing,passing,passing


In [36]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
math,12.0,85.583333,11.007917,62.0,77.75,85.5,95.25,99.0
english,12.0,82.000000,8.696917,61.0,78.50,84.5,88.00,91.0
reading,12.0,83.333333,12.928709,60.0,72.75,89.0,93.25,98.0
